<a href="https://colab.research.google.com/github/VastSea0/nasa-hackathon/blob/main/helo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Kütüphaneler ve temel setup
!pip install earthaccess xarray cartopy matplotlib numpy

import os
import sys
import traceback
from datetime import datetime
import json

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

try:
    import earthaccess
except Exception:
    print("ERROR: earthaccess import failed. earthaccess configure yap.")
    raise

try:
    from google import genai
    from google.genai import types
except Exception:
    genai = None
    types = None

# Config
BBOX = (26, 36, 45, 42)
DATES = ("2025-09-01", "2025-10-02")
TIME_INDEX = 0
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

earthaccess.login()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 46.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.

In [2]:
def search_and_open(short_name, dates=DATES, bbox=BBOX, fail_on_empty=True):
    print(f"[INFO] Searching {short_name} for {dates} @ bbox={bbox} ...")
    results = earthaccess.search_data(short_name=short_name, temporal=dates, bounding_box=bbox, cloud_hosted=True)
    if not results:
        if fail_on_empty:
            raise RuntimeError(f"No results for {short_name} {dates} {bbox}")
        return None
    files = earthaccess.open(results[0:1])
    if not files:
        raise RuntimeError(f"earthaccess.open returned no file objects for {short_name}")
    ds = xr.open_dataset(files[0])
    print(f"[INFO] Opened dataset {short_name}. Variables: {list(ds.variables.keys())[:40]} ...")
    return ds

def safe_var(ds, names):
    for n in names:
        if n in ds.variables:
            return ds[n]
    raise KeyError(f"None of {names} found. Available: {list(ds.variables.keys())[:80]}")

def subset_time_space(da, time_index=TIME_INDEX, bbox=BBOX):
    lon_min, lat_min, lon_max, lat_max = bbox
    if 'lon' in da.coords:
        lon = da.coords['lon']
        try:
            if float(lon.max()) > 180:
                da = da.assign_coords(lon=(((da.lon + 180) % 360) - 180)).sortby('lon')
        except Exception:
            pass
    if 'time' in da.dims:
        da = da.isel(time=time_index)
    return da.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))


In [10]:
def build_and_plot_map(dates=DATES, savepath=None):
    print("[STEP] Fetching datasets...")
    ds_atm = search_and_open("M2T1NXSLV", dates=dates)
    ds_flx = search_and_open("M2T1NXFLX", dates=dates, fail_on_empty=False)
    ds_lnd = search_and_open("M2T1NXLND", dates=dates, fail_on_empty=False)
    ds_aer = search_and_open("M2T1NXAER", dates=dates, fail_on_empty=False)

    # Variables
    precip = None
    if ds_flx is not None:
        for cand in ["PRECTOT", "PRECTOTCORR", "PRATE", "PRECIP", "PRECC"] :
            if cand in ds_flx.variables:
                precip = ds_flx[cand]
                break
    temp = safe_var(ds_atm, ["T2M", "TMP2m", "TEMP_2M", "T2MDEW", "T10M"])
    u_wind = None
    v_wind = None
    for u_c in ["U10M", "U2M", "U_10M", "U10", "U10M_AV"]:
        if u_c in ds_atm.variables:
            u_wind = ds_atm[u_c]; break
    for v_c in ["V10M", "V2M", "V_10M", "V10", "V10M_AV"]:
        if v_c in ds_atm.variables:
            v_wind = ds_atm[v_c]; break
    soil = None
    if ds_lnd is not None:
        for cand in ["GWETROOT", "GWETPROF", "GWETTOP", "SOILM", "SMROOT"]:
            if cand in ds_lnd.variables:
                soil = ds_lnd[cand]; break
    aerosol = None
    if ds_aer is not None:
        for cand in ["TOTEXTTAU", "AOD", "AOD550", "DUEXTTAU", "DUCMASS"]:
            if cand in ds_aer.variables:
                aerosol = ds_aer[cand]; break

    # Subset
    precip_s = subset_time_space(precip) if precip is not None else None
    temp_s = subset_time_space(temp) - 273.15
    u_s = subset_time_space(u_wind) if u_wind is not None else None
    v_s = subset_time_space(v_wind) if v_wind is not None else None
    soil_s = subset_time_space(soil) if soil is not None else None
    aero_s = subset_time_space(aerosol) if aerosol is not None else None

    # Derived
    wind_speed = np.sqrt(u_s**2 + v_s**2) if (u_s is not None and v_s is not None) else None
    if soil_s is not None:
        soil_clim = float(np.nanmean(soil_s))
        drought_index = 1.0 - (soil_s / (soil_clim + 1e-9))
        drought_index = drought_index.clip(min=0.0, max=2.0)
        drought_index = (drought_index - float(drought_index.min())) / (float(drought_index.max()) - float(drought_index.min()) + 1e-9)
    elif precip_s is not None:
        precip_mean = float(np.nanmean(precip_s))
        drought_index = 1.0 - (precip_s / (precip_mean + 1e-9))
        drought_index = drought_index.clip(min=0.0, max=2.0)
        drought_index = (drought_index - float(drought_index.min())) / (float(drought_index.max()) - float(drought_index.min()) + 1e-9)
    else:
        drought_index = xr.zeros_like(temp_s) * 0.0

    precip_mm_day = precip_s * 86400.0 if precip_s is not None else None

    # Plot
    print("[STEP] Plotting combined map ...")
    fig = plt.figure(figsize=(12, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([BBOX[0], BBOX[2], BBOX[1], BBOX[3]], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE, linewidth=0.6)
    ax.add_feature(cfeature.BORDERS, linewidth=0.6)
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.6, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    if precip_mm_day is not None:
        im1 = precip_mm_day.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="Blues", alpha=0.85, add_colorbar=False)
        plt.colorbar(im1, ax=ax, fraction=0.037, pad=0.02).set_label("Precipitation (mm/day)")
    im2 = drought_index.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="Reds", alpha=0.38, add_colorbar=False)
    plt.colorbar(im2, ax=ax, fraction=0.037, pad=0.1).set_label("Drought index (0..1)")
    temp_s.plot.contour(ax=ax, transform=ccrs.PlateCarree(), colors='k', linewidths=0.6, add_colorbar=False)
    if u_s is not None and v_s is not None:
        skip = (slice(None, None, 6), slice(None, None, 6))
        ax.quiver(u_s["lon"].values[skip[1]], u_s["lat"].values[skip[0]],
                  u_s.values[skip], v_s.values[skip], transform=ccrs.PlateCarree(), color='gray', scale=300)
    if aero_s is not None:
        aero_s.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="YlGnBu_r", alpha=0.25, add_colorbar=False)
    outpath = os.path.join(OUTPUT_DIR, f"combined_map_{datetime.utcnow().strftime('%Y%m%dT%H%M%S')}.png")
    plt.savefig(outpath, dpi=220, bbox_inches='tight')
    plt.close(fig)

    # Summary
    summary = {
        'map_path': outpath,
        'precip_mean_mm_per_day': float(np.nanmean(precip_mm_day)) if precip_mm_day is not None else None,
        'temp_mean_C': float(np.nanmean(temp_s)),
        'wind_mean_m_s': float(np.nanmean(wind_speed)) if wind_speed is not None else None,
        'drought_index_mean': float(np.nanmean(drought_index)),
        'aod_mean': float(np.nanmean(aero_s)) if aero_s is not None else None
    }
    print("[DONE] analysis summary:", json.dumps(summary, indent=2))
    return summary

In [6]:
def call_gemini_with_analysis(analysis_dict, model_name="gemini-2.5-flash-lite", thinking_budget=0):
    if genai is None or types is None:
        print("[WARN] google-genai not installed/importable. Skipping Gemini call.")
        return None
    api_key = "skillissue"
    if not api_key:
        print("[WARN] GEMINI_API_KEY not set. Skipping Gemini call.")
        return None

    client = genai.Client(api_key=api_key)
    user_text = f"""Numeric analysis for bbox={analysis_dict.get('bbox')} dates={analysis_dict.get('dates')}:
{json.dumps(analysis_dict, indent=2)}
Please produce: 1) 3-sentence summary 2) risk bullets (agri/health/transport) 3) 3 actionable recommendations. Format as JSON with keys: summary, risks, recommendations.
"""
    contents = [ types.Content(role="user", parts=[ types.Part.from_text(text=user_text) ]) ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(thinking_budget=thinking_budget),
        tools = []
    )

    print("[LLM] Sending to Gemini:", model_name)
    collected = ""
    for chunk in client.models.generate_content_stream(model=model_name, contents=contents, config=generate_content_config):
        text = getattr(chunk, "text", None)
        if text:
            print(text, end="", flush=True)
            collected += text
    print("\n[LLM] Done.")
    return collected


In [13]:
start_date_str = input("Lütfen başlangıç tarihini girin (YYYY-MM-DD formatında): ")
end_date_str = input("Lütfen bitiş tarihini girin (YYYY-MM-DD formatında): ")
user_dates = (start_date_str, end_date_str)

summary = build_and_plot_map(dates=user_dates)

# Eğer Gemini kullanmak istersen:
llm_text = call_gemini_with_analysis(summary)
print(llm_text)

Lütfen başlangıç tarihini girin (YYYY-MM-DD formatında): 2024-10-4
Lütfen bitiş tarihini girin (YYYY-MM-DD formatında): 2025-10-4
[STEP] Fetching datasets...
[INFO] Searching M2T1NXSLV for ('2024-10-4', '2025-10-4') @ bbox=(26, 36, 45, 42) ...


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Opened dataset M2T1NXSLV. Variables: ['CLDPRS', 'CLDTMP', 'DISPH', 'H1000', 'H250', 'H500', 'H850', 'OMEGA500', 'PBLTOP', 'PS', 'Q250', 'Q500', 'Q850', 'QV10M', 'QV2M', 'SLP', 'T10M', 'T250', 'T2M', 'T2MDEW', 'T2MWET', 'T500', 'T850', 'TO3', 'TOX', 'TQI', 'TQL', 'TQV', 'TROPPB', 'TROPPT', 'TROPPV', 'TROPQ', 'TROPT', 'TS', 'U10M', 'U250', 'U2M', 'U500', 'U50M', 'U850'] ...
[INFO] Searching M2T1NXFLX for ('2024-10-4', '2025-10-4') @ bbox=(26, 36, 45, 42) ...


...

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Opened dataset M2T1NXAER. Variables: ['BCANGSTR', 'BCCMASS', 'BCEXTTAU', 'BCFLUXU', 'BCFLUXV', 'BCSCATAU', 'BCSMASS', 'DMSCMASS', 'DMSSMASS', 'DUANGSTR', 'DUCMASS', 'DUCMASS25', 'DUEXTT25', 'DUEXTTAU', 'DUFLUXU', 'DUFLUXV', 'DUSCAT25', 'DUSCATAU', 'DUSMASS', 'DUSMASS25', 'OCANGSTR', 'OCCMASS', 'OCEXTTAU', 'OCFLUXU', 'OCFLUXV', 'OCSCATAU', 'OCSMASS', 'SO2CMASS', 'SO2SMASS', 'SO4CMASS', 'SO4SMASS', 'SSANGSTR', 'SSCMASS', 'SSCMASS25', 'SSEXTT25', 'SSEXTTAU', 'SSFLUXU', 'SSFLUXV', 'SSSCAT25', 'SSSCATAU'] ...
[STEP] Plotting combined map ...


/tmp/ipython-input-3649923149.py:83: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  outpath = os.path.join(OUTPUT_DIR, f"combined_map_{datetime.utcnow().strftime('%Y%m%dT%H%M%S')}.png")


[DONE] analysis summary: {
  "map_path": "output/combined_map_20251004T203723.png",
  "precip_mean_mm_per_day": 0.004392288625240326,
  "temp_mean_C": 13.763850212097168,
  "wind_mean_m_s": 2.2573463916778564,
  "drought_index_mean": 0.18227094411849976,
  "aod_mean": 0.06812472641468048
}
[LLM] Sending to Gemini: gemini-2.5-flash-lite
```json
{
  "summary": "Analysis of the provided data reveals a low mean precipitation of 0.004 mm/day, a moderate average temperature of 13.76°C, and a light average wind speed of 2.26 m/s. The mean drought index is low at 0.18, suggesting minimal drought conditions.  Additionally, the mean Aerosol Optical Depth (AOD) is 0.07, indicating relatively clean air.",
  "risks": {
    "agri": [
      "Low precipitation could stress rain-fed crops, requiring supplemental irrigation.",
      "Moderate temperatures are generally favorable, but prolonged dry spells may lead to water scarcity for irrigation needs."
    ],
    "health": [
      "The low AOD suggests

# Task
Kullanıcının kendi tarih aralığını seçmesine izin veren ve sonuçları web arayüzünde gösteren bir uygulama oluşturun.

## Web uygulaması çerçevesi seçimi

### Subtask:
Flask veya Django gibi bir Python web çerçevesi seçin.


## Notebook kodunu modüler hale getirme

### Subtask:
Notebook'taki fonksiyonları (veri çekme, analiz, çizim) ayrı Python dosyalarına taşıyarak modüler hale getirin.


**Reasoning**:
Create a new Python file `weather_utils.py` and move the specified functions and constants into it, including the necessary imports.



In [14]:
# weather_utils.py

import os
import sys
import traceback
from datetime import datetime
import json

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

try:
    import earthaccess
except Exception:
    print("ERROR: earthaccess import failed. earthaccess configure yap.")
    raise

try:
    from google import genai
    from google.genai import types
except Exception:
    genai = None
    types = None

# Config
BBOX = (26, 36, 45, 42)
DATES = ("2025-09-01", "2025-10-02")
TIME_INDEX = 0
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# earthaccess.login() # This should probably be handled in the main app

def search_and_open(short_name, dates=DATES, bbox=BBOX, fail_on_empty=True):
    print(f"[INFO] Searching {short_name} for {dates} @ bbox={bbox} ...")
    results = earthaccess.search_data(short_name=short_name, temporal=dates, bounding_box=bbox, cloud_hosted=True)
    if not results:
        if fail_on_empty:
            raise RuntimeError(f"No results for {short_name} {dates} {bbox}")
        return None
    files = earthaccess.open(results[0:1])
    if not files:
        raise RuntimeError(f"earthaccess.open returned no file objects for {short_name}")
    ds = xr.open_dataset(files[0])
    print(f"[INFO] Opened dataset {short_name}. Variables: {list(ds.variables.keys())[:40]} ...")
    return ds

def safe_var(ds, names):
    for n in names:
        if n in ds.variables:
            return ds[n]
    raise KeyError(f"None of {names} found. Available: {list(ds.variables.keys())[:80]}")

def subset_time_space(da, time_index=TIME_INDEX, bbox=BBOX):
    lon_min, lat_min, lon_max, lat_max = bbox
    if 'lon' in da.coords:
        lon = da.coords['lon']
        try:
            if float(lon.max()) > 180:
                da = da.assign_coords(lon=(((da.lon + 180) % 360) - 180)).sortby('lon')
        except Exception:
            pass
    if 'time' in da.dims:
        da = da.isel(time=time_index)
    return da.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))

def build_and_plot_map(dates=DATES, savepath=None):
    print("[STEP] Fetching datasets...")
    ds_atm = search_and_open("M2T1NXSLV", dates=dates)
    ds_flx = search_and_open("M2T1NXFLX", dates=dates, fail_on_empty=False)
    ds_lnd = search_and_open("M2T1NXLND", dates=dates, fail_on_empty=False)
    ds_aer = search_and_open("M2T1NXAER", dates=dates, fail_on_empty=False)

    # Variables
    precip = None
    if ds_flx is not None:
        for cand in ["PRECTOT", "PRECTOTCORR", "PRATE", "PRECIP", "PRECC"] :
            if cand in ds_flx.variables:
                precip = ds_flx[cand]
                break
    temp = safe_var(ds_atm, ["T2M", "TMP2m", "TEMP_2M", "T2MDEW", "T10M"])
    u_wind = None
    v_wind = None
    for u_c in ["U10M", "U2M", "U_10M", "U10", "U10M_AV"]:
        if u_c in ds_atm.variables:
            u_wind = ds_atm[u_c]; break
    for v_c in ["V10M", "V2M", "V_10M", "V10", "V10M_AV"]:
        if v_c in ds_atm.variables:
            v_wind = ds_atm[v_c]; break
    soil = None
    if ds_lnd is not None:
        for cand in ["GWETROOT", "GWETPROF", "GWETTOP", "SOILM", "SMROOT"]:
            if cand in ds_lnd.variables:
                soil = ds_lnd[cand]; break
    aerosol = None
    if ds_aer is not None:
        for cand in ["TOTEXTTAU", "AOD", "AOD550", "DUEXTTAU", "DUCMASS"]:
            if cand in ds_aer.variables:
                aerosol = ds_aer[cand]; break

    # Subset
    precip_s = subset_time_space(precip) if precip is not None else None
    temp_s = subset_time_space(temp) - 273.15
    u_s = subset_time_space(u_wind) if u_wind is not None else None
    v_s = subset_time_space(v_wind) if v_wind is not None else None
    soil_s = subset_time_space(soil) if soil is not None else None
    aero_s = subset_time_space(aerosol) if aerosol is not None else None

    # Derived
    wind_speed = np.sqrt(u_s**2 + v_s**2) if (u_s is not None and v_s is not None) else None
    if soil_s is not None:
        soil_clim = float(np.nanmean(soil_s))
        drought_index = 1.0 - (soil_s / (soil_clim + 1e-9))
        drought_index = drought_index.clip(min=0.0, max=2.0)
        drought_index = (drought_index - float(drought_index.min())) / (float(drought_index.max()) - float(drought_index.min()) + 1e-9)
    elif precip_s is not None:
        precip_mean = float(np.nanmean(precip_s))
        drought_index = 1.0 - (precip_s / (precip_mean + 1e-9))
        drought_index = drought_index.clip(min=0.0, max=2.0)
        drought_index = (drought_index - float(drought_index.min())) / (float(drought_index.max()) - float(drought_index.min()) + 1e-9)
    else:
        drought_index = xr.zeros_like(temp_s) * 0.0

    precip_mm_day = precip_s * 86400.0 if precip_s is not None else None

    # Plot
    print("[STEP] Plotting combined map ...")
    fig = plt.figure(figsize=(12, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([BBOX[0], BBOX[2], BBOX[1], BBOX[3]], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE, linewidth=0.6)
    ax.add_feature(cfeature.BORDERS, linewidth=0.6)
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.6, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    if precip_mm_day is not None:
        im1 = precip_mm_day.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="Blues", alpha=0.85, add_colorbar=False)
        plt.colorbar(im1, ax=ax, fraction=0.037, pad=0.02).set_label("Precipitation (mm/day)")
    im2 = drought_index.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="Reds", alpha=0.38, add_colorbar=False)
    plt.colorbar(im2, ax=ax, fraction=0.037, pad=0.1).set_label("Drought index (0..1)")
    temp_s.plot.contour(ax=ax, transform=ccrs.PlateCarree(), colors='k', linewidths=0.6, add_colorbar=False)
    if u_s is not None and v_s is not None:
        skip = (slice(None, None, 6), slice(None, None, 6))
        ax.quiver(u_s["lon"].values[skip[1]], u_s["lat"].values[skip[0]],
                  u_s.values[skip], v_s.values[skip], transform=ccrs.PlateCarree(), color='gray', scale=300)
    if aero_s is not None:
        aero_s.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="YlGnBu_r", alpha=0.25, add_colorbar=False)
    outpath = os.path.join(OUTPUT_DIR, f"combined_map_{datetime.utcnow().strftime('%Y%m%dT%H%M%S')}.png")
    plt.savefig(outpath, dpi=220, bbox_inches='tight')
    plt.close(fig)

    # Summary
    summary = {
        'map_path': outpath,
        'precip_mean_mm_per_day': float(np.nanmean(precip_mm_day)) if precip_mm_day is not None else None,
        'temp_mean_C': float(np.nanmean(temp_s)),
        'wind_mean_m_s': float(np.nanmean(wind_speed)) if wind_speed is not None else None,
        'drought_index_mean': float(np.nanmean(drought_index)),
        'aod_mean': float(np.nanmean(aero_s)) if aero_s is not None else None,
        'bbox': BBOX,
        'dates': dates
    }
    print("[DONE] analysis summary:", json.dumps(summary, indent=2))
    return summary

def call_gemini_with_analysis(analysis_dict, model_name="gemini-2.5-flash-lite", thinking_budget=0):
    if genai is None or types is None:
        print("[WARN] google-genai not installed/importable. Skipping Gemini call.")
        return None
    api_key = "skillissue" # Placeholder, replace with actual key retrieval
    if not api_key:
        print("[WARN] GEMINI_API_KEY not set. Skipping Gemini call.")
        return None

    client = genai.Client(api_key=api_key)
    user_text = f"""Numeric analysis for bbox={analysis_dict.get('bbox')} dates={analysis_dict.get('dates')}:
{json.dumps(analysis_dict, indent=2)}
Please produce: 1) 3-sentence summary 2) risk bullets (agri/health/transport) 3) 3 actionable recommendations. Format as JSON with keys: summary, risks, recommendations.
"""
    contents = [ types.Content(role="user", parts=[ types.Part.from_text(text=user_text) ]) ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(thinking_budget=thinking_budget),
        tools = []
    )

    print("[LLM] Sending to Gemini:", model_name)
    collected = ""
    for chunk in client.models.generate_content_stream(model=model_name, contents=contents, config=generate_content_config):
        text = getattr(chunk, "text", None)
        if text:
            print(text, end="", flush=True)
            collected += text
    print("\n[LLM] Done.")
    return collected
